# imports

In [1]:
import sys
sys.path.append('../')

import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

import json
import numpy as np
from models.patch_to_score.dataset import PatchToScoreCrossValidationDataset
from models.patch_to_score.models.modules.transformer_encoder_mlp import TransformerEncoderMLP
from models.patch_to_score.models.modules.global_sum_pooling import GlobalSumPooling

2025-09-17 13:19:33.655910: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-17 13:19:33.769669: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-17 13:19:34.742112: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/iscb/wolfson/doririmon/anaconda3/

# constants

In [2]:
model_results_dir_path = '/home/iscb/wolfson/doririmon/home/order/ubinet/repo/ubinet/results/patch_to_score/hypotheses/pts_shalllow_mlps_autoregressive_scannet/21_07/2025-08-08_22693c7950'
fold = 0

# load model

In [3]:
with open(f'{model_results_dir_path}/configuration.json', 'rb') as f:
    configuration = json.load(f)
model_configuration = configuration['model']

In [4]:
from models.patch_to_score.bootstrappers.model import build_model_from_configuration

In [5]:
model = build_model_from_configuration(**model_configuration)
model.load_weights(f'{model_results_dir_path}/fold_{fold}/model.keras')

2025-09-17 13:19:39.939330: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


initializing projection for skip connection: 11 -> 512


# analysis

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 hot_encoded_value_input (Input  [(None, 11)]        0           []                               
 Layer)                                                                                           
                                                                                                  
 tf.math.argmax (TFOpLambda)    (None,)              0           ['hot_encoded_value_input[0][0]']
                                                                                                  
 tf.__operators__.getitem (Slic  (None, 1)           0           ['tf.math.argmax[0][0]']         
 ingOpLambda)                                                                                     
                                                                                              

# inference

In [7]:
cross_validation_dataset = PatchToScoreCrossValidationDataset(**configuration['data'])
dataset = cross_validation_dataset.fold_datasets[fold]
batch = [*dataset.validation_set, dataset.validation_sizes, dataset.validation_num_patch]

folds amount: 5


In [8]:
predictions = model(batch)
saved_predictions = np.load(f'{model_results_dir_path}/fold_{fold}/validation/predictions.npy')

In [9]:
np.allclose(predictions, saved_predictions)

True